# Chapter 7: Teaching by Example (Few-Shot)

**Scenario:** You have a new intern (the AI). You want them to answer customer emails in a specific, quirky style. You could write a 10-page style guide, OR you could just show them 3 previous emails you wrote and say "Do it like this".

In this notebook, you'll learn:
- 0️⃣ **Zero-Shot**: Asking without examples (the default).
- 3️⃣ **Few-Shot**: Proving examples to guide tone and format.
- 🎨 **Style Transfer**: Making the model sound like a pirate, a lawyer, or a soothing parent just by showing it how.

---

## 1. Setup

Standard setup.

In [1]:
!pip install -q litellm python-dotenv

import os
from dotenv import load_dotenv
from litellm import completion
import litellm
import logging

# Suppress noisy debug logs
litellm.suppress_debug_info = True
logging.getLogger("litellm").setLevel(logging.CRITICAL)

load_dotenv()
MODEL_NAME = os.getenv('DEFAULT_MODEL', 'gemini/gemini-2.5-flash')

In [2]:
def get_completion(prompt, system_prompt=None, prefill=None, temperature=0.0, max_tokens=1024):
    messages = []
    if system_prompt:
        messages.append({"role": "system", "content": system_prompt})
    messages.append({"role": "user", "content": prompt})
    
    if prefill:
        messages.append({"role": "assistant", "content": prefill})
    
    response = completion(
        model=MODEL_NAME,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens
    )
    
    if prefill:
        return prefill + response.choices[0].message.content
    return response.choices[0].message.content

---

## 2. Zero-Shot vs. Few-Shot

**Zero-Shot** is what we've done mostly so far: giving an instruction without examples.

Example: "Will Santa bring me presents?"

In [3]:
print(get_completion("Will Santa bring me presents on Christmas?"))

Whether or not Santa will bring you presents on Christmas depends on a few things! Traditionally, Santa brings presents to children who have been good throughout the year. So, have you been trying your best to be kind, helpful, and well-behaved?

Also, do you believe in Santa? A little bit of belief can go a long way!

Ultimately, I can't say for sure, but I hope you have a wonderful Christmas, regardless!



The answer is likely factual and boring. But if we're building a "Parent Bot", we want magic.

**Few-Shot** means giving examples of the input and the desired output.

In [4]:
prompt = """
Please complete the conversation by writing the next line, speaking as "A".

Q: Is the tooth fairy real?
A: Of course, sweetie. Wrap up your tooth and put it under your pillow tonight. There might be something waiting for you in the morning.

Q: Will Santa bring me presents on Christmas?
"""

print(get_completion(prompt))

A: Absolutely! If you're a good little boy/girl, Santa will definitely be visiting on Christmas Eve.



By seeing just ONE example of how "A" speaks (warm, encouraging, keeping the magic alive), the model mimics it perfectly.

## 3. Teaching Formatting

Few-shot is also the best way to teach complex output formats without writing a regex manual.

In [5]:
prompt = """
Extract the person and their profession from the text.

Text: Dr. Liam Patel, a neurosurgeon who revolutionized surgical techniques.
Output: 1. Dr. Liam Patel [NEUROSURGEON]

Text: Olivia Chen was an innovative architect who transformed the village.
Output: 2. Olivia Chen [ARCHITECT]

Text: Chef Oliver Hamilton has transformed the culinary scene with his farm-to-table restaurant.
Output:
"""

print(get_completion(prompt))

3. Oliver Hamilton [CHEF]



---

## 4. Exercises

### Exercise 1: The Email Classifier (Again)

Remember our Email Classifier? It struggled to format the output as just the letter "(B)".

**Goal:** Use Few-Shot prompting to teach it EXACTLY how to respond. 
Provide 3 examples of inputs and outputs in the `PROMPT` variable below.

In [6]:
email_to_classify = "My Mixmaster4000 is smoking. It smells like burning rubber."

PROMPT = f"""
Classify emails into: (A) Pre-sale, (B) Broken, (C) Billing, (D) Other.

Email: Can I paint with this mixer?
Class: (A)

Email: I want a refund, you charged me twice!
Class: (C)

Email: How do I turn it on?
Class: (D)

Email: {email_to_classify}
Class:
"""

response = get_completion(PROMPT)
print(response)

# Grading
if "(B)" in response:
    print("✅ Classification Correct!")
else:
    print("❌ Try again.")

Class: (B)

✅ Classification Correct!


---

## Summary

When instructions fail, **use examples**.
- It's often faster than writing explanations.
- It captures "soft" requirements like tone and style.
- It enforces strict formatting compliance.

In the next chapter, we'll learn how to stop the model from making things up (**Hallucinations**).